In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import math

import numpy as np
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
import torch
import pandas as pd
import numpy as np

from model import TDNN

In [2]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, data_df, target_df):
        
        data = pd.read_csv(data_df)
        target = pd.read_csv(target_df)
    
    self.data = 

    def __len__(self):
        return len()

    def __getitem__(self,idx):
        return

hello world
